<a href="https://colab.research.google.com/github/Kraken2003/Lofi-Gen/blob/main/LofiGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import os
from music21 import converter, instrument, note, chord
from music21 import stream
import numpy as np
import keras
from keras.utils import to_categorical
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Dropout, Flatten
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint
import pandas as pd

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:

# Specify the folder where the pickle files are stored
folder_path = r'/content/drive/MyDrive/lofimidi'

# Get a list of all files in the specified folder
pickle_file_names = [file for file in os.listdir(folder_path) if file.endswith(".pkl")]

# Initialize an empty list to store the loaded data
notes = []

# Load data from each pickle file and append it to the notes list
for file_name in pickle_file_names:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        notes.extend(data)

print(len(notes))


125825


In [4]:
notes[0:10]

['11.4', 'G#4', 'E4', 'B3', 'G#4', 'E4', 'B3', '9.1.4', '9.2', '9.2']

In [5]:
def process_notes(notes):

    # Look at 50 previous notes to make a prediction
    #   We can tune this parameter if needed, based on the length of
    #   chord progressions
    seqLength = 50
    print('Using sequence length of {}'.format(seqLength))

    pitchSet = sorted(set(notes))
    numPitches = len(pitchSet)
    # here pitches are either notes or chords
    #   they are sorted lexicographically, so a chord 'C4.E4' will come after a
    #   note 'C4'
    print('Identified {} pitches'.format(numPitches))

    # Map each note/chord to a number normalized to (0,1)
    pitchMapping = dict((note, number) for (number, note) in enumerate(pitchSet))

    networkInput = []
    networkOutput = []

    print('Starting sequencing of {} notes'.format(len(notes)))
    for i in range(0, len(notes)- seqLength):
        sequenceIn = notes[i:i+seqLength]
        predictionOut = notes[i+seqLength]

        networkInput.append([pitchMapping[note] for note in sequenceIn])
        networkOutput.append(pitchMapping[predictionOut])

        if (i+1) % 50000 == 0:
            print('Finished making {} sequences'.format(i+1))

    networkInput = np.array(networkInput)
    networkOutput = np.array(networkOutput)

    numSeqs = len(networkInput)
    # reshape input to match the LSTM layer format
    networkInputShaped = np.reshape(networkInput, (numSeqs, seqLength, 1))
    networkInputShaped = networkInputShaped / numPitches

    networkOutputShaped = to_categorical(networkOutput)

    return networkInput, networkOutput, networkInputShaped, networkOutputShaped, numPitches

In [13]:
def create_model(networkInputShaped,networkOutputShaped,numPitches,num_epochs=30):
    model = Sequential()
    model.add(Dropout(0.2))
    model.add(LSTM(
        512,
        input_shape=(networkInputShaped.shape[1], networkInputShaped.shape[2]),
        return_sequences=True
    ))
    model.add(Dense(256))
    model.add(Dense(256))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dense(256))
    model.add(LSTM(512))
    model.add(Dense(numPitches))
    model.add(Dense(numPitches))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


    filepath = r'/content/drive/MyDrive/lofimidi/model_weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger_1.hdf5'
    checkpoint = ModelCheckpoint(
        filepath, monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]


    history = model.fit(networkInputShaped, networkOutputShaped, epochs=num_epochs, batch_size=64, callbacks=callbacks_list)

    return model, history

In [16]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    # Selects a random row from the network_input
    start = np.random.randint(0, len(network_input)-1)
    print(f'start: {start}')
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Random row from network_input
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        if (note_index+1)%100 == 0:
          print(note_index)
        # Reshapes pattern into a vector
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        # Standarizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Outputs a OneHot encoded vector, so this picks the columns
        # with the highest probability
        index = np.argmax(prediction)
        # Maps the note to its respective index
        result = int_to_note[index]
        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = np.append(pattern,index)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [8]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='output.mid')

In [9]:
n_vocab = len(set(notes))
print(n_vocab)

448


In [10]:
pitchnames = sorted(set(item for item in notes))
print(len(pitchnames))

448


In [11]:
networkInput, networkOutput, networkInputShaped, networkOutputShaped, numPitches = process_notes(notes)

Using sequence length of 50
Identified 448 pitches
Starting sequencing of 125825 notes
Finished making 50000 sequences
Finished making 100000 sequences


In [12]:
numPitches

448

In [14]:
model, history = create_model(networkInputShaped,networkOutputShaped,numPitches,num_epochs=50)

Epoch 1/50
1966/1966 [==============================] - ETA: 0s - loss: 4.6500 - accuracy: 0.0389
Epoch 1: loss improved from inf to 4.65003, saving model to /content/drive/MyDrive/lofimidi/model_weights/weights-improvement-01-4.6500-bigger_1.hdf5
1966/1966 [==============================] - 88s 40ms/step - loss: 4.6500 - accuracy: 0.0389


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/50
1966/1966 [==============================] - ETA: 0s - loss: 4.1945 - accuracy: 0.0918
Epoch 2: loss improved from 4.65003 to 4.19455, saving model to /content/drive/MyDrive/lofimidi/model_weights/weights-improvement-02-4.1945-bigger_1.hdf5
1966/1966 [==============================] - 81s 41ms/step - loss: 4.1945 - accuracy: 0.0918
Epoch 3/50
1965/1966 [============================>.] - ETA: 0s - loss: 3.6693 - accuracy: 0.1660
Epoch 3: loss improved from 4.19455 to 3.66934, saving model to /content/drive/MyDrive/lofimidi/model_weights/weights-improvement-03-3.6693-bigger_1.hdf5
1966/1966 [==============================] - 80s 41ms/step - loss: 3.6693 - accuracy: 0.1660
Epoch 4/50
1966/1966 [==============================] - ETA: 0s - loss: 3.2626 - accuracy: 0.2286
Epoch 4: loss improved from 3.66934 to 3.26262, saving model to /content/drive/MyDrive/lofimidi/model_weights/weights-improvement-04-3.2626-bigger_1.hdf5
1966/1966 [==============================] - 81s 41ms/step

In [17]:
prediction_output = generate_notes(model, networkInputShaped, pitchnames, n_vocab)
create_midi(prediction_output)

start: 17984
99
199
299
399
499
